# 2.2 Task 1: Evaluation of the word2vec-google-news-300 Pre-trained Model

In [1]:
import gensim.downloader as api

model_word2vec_google_news_300 = api.load("word2vec-google-news-300")

[=================================================-] 99.8% 1659.9/1662.8MB downloaded


In [2]:
def documentation(model):
    # Let's print all the methods of the model
    print(dir(model))

documentation(model_word2vec_google_news_300)

['__class__', '__contains__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_adapt_by_suffix', '_load_specials', '_log_evaluate_word_analogies', '_save_specials', '_smart_save', '_upconvert_old_d2vkv', '_upconvert_old_vocab', 'add_lifecycle_event', 'add_vector', 'add_vectors', 'allocate_vecattrs', 'closer_than', 'cosine_similarities', 'distance', 'distances', 'doesnt_match', 'evaluate_word_analogies', 'evaluate_word_pairs', 'expandos', 'fill_norms', 'get_index', 'get_normed_vectors', 'get_vecattr', 'get_vector', 'has_index_for', 'index2entity', 'index2word', 'index_to_key', 'init_sims', 'intersect_word2vec_format', 'key_to_index', 'lifecycle_events', 'load', 'load_

In [3]:
def testing(model): 
    print("Let's do some testing to explore how different methods work!\n")
    
    question_word = "enormously"
    options = ["appropriately", "uniquely", "tremendously", "decidedly"]
    print(f"Let's test the following question_word to find the closest synonym: '{question_word}'")
    print(f"Given the following options: {options}")
    
    # Let's test the "most_similar_to_given" method!
    answer_word = model.most_similar_to_given(question_word, options)
    print(f"\nUsing the 'model.most_similar_to_given' method, we compute the following: '{answer_word}'")
    
    # Let's test the "model.similarity" method!
    print(f"\nLet's see the cosine between the '{question_word}' and '{answer_word}' using the 'model.similarity' method!")
    cos_sim = model.similarity(question_word, answer_word)
    print(f"The cosine similarity between {question_word} and {answer_word} is: {cos_sim}")

    # Let's make sure the answer computed above correctly calculates the cosine similarity! 
    from numpy import dot
    from numpy.linalg import norm
    v1 = model.get_vector(question_word)
    v2 = model.get_vector(answer_word)
    cos_sim = dot(v1, v2)/(norm(v1)*norm(v2))
    print(f"\nIf we compute the cosine similarity manually, we should also achieve the same result: {cos_sim}")

testing(model_word2vec_google_news_300)

Let's do some testing to explore how different methods work!

Let's test the following question_word to find the closest synonym: 'enormously'
Given the following options: ['appropriately', 'uniquely', 'tremendously', 'decidedly']

Using the 'model.most_similar_to_given' method, we compute the following: 'tremendously'

Let's see the cosine between the 'enormously' and 'tremendously' using the 'model.similarity' method!
The cosine similarity between enormously and tremendously is: 0.8185791969299316

If we compute the cosine similarity manually, we should also achieve the same result: 0.8185791969299316


In [4]:
import csv
from enum import Enum
import random

class Labels(Enum):
    GUESS = "guess"
    CORRECT = "correct"
    WRONG = "wrong"
    
def random_guess(options):
    random_index = random.randint(0, len(options) - 1)
    return options[random_index]

def print_analysis(model, model_name, num_correct, num_wrong):
    print(f"\n")
    print(f"(a) Model name: {model_name}")
    print(f"(b) The size of the vocabulary: {len(model)}")
    print(f"(c) The number of correct labels (call this 'C'): {num_correct}")
    print(f"(d) The number of questions the Model answered without guessing (call this 'V' ): {num_correct + num_wrong}")
    print(f"(e) the accuracy of the model (i.e. C/V): {num_correct/(num_correct + num_wrong)}")

def compute_most_similar_term(model, question_word, options):
    similarity = None
    guess_word = None
    for option in options:
        try:
            current = model.similarity(question_word, option)
            if not similarity or current > similarity:
                similarity = current
                guess_word = option
        except KeyError:
            pass     
    return (guess_word, similarity)

def compute_synonym(model, row):
    question_word = row[0]
    answer_word = row[1]
    options = row[2:]
    
    guess_word, similarity, label = None, None, None
    
    # Let's check if the question word is defined in the vocabulary
    try:
        question_word_vector = model.get_vector(question_word)
    except KeyError:
        label = Labels.GUESS
        guess_word = random_guess(options)
        return (question_word, answer_word, guess_word, similarity, Labels.GUESS)
    
    # If the question word is defined, we can proceed with finding the most similar term
    (guess_word, similarity) = compute_most_similar_term(model, question_word, options)
    
    if guess_word == answer_word:
        label = Labels.CORRECT
    elif not guess_word:
        # If none of the options are in the vocabulary, then it's a guess
        label = Labels.GUESS
        guess_word = random_guess(options)
    else:
        label = Labels.WRONG
    
    return (question_word, answer_word, guess_word, similarity, label)
            
def compute_synonyms(model, model_name):
    with open('synonyms.csv', newline='') as csvfile:

        reader = csv.reader(csvfile, delimiter=',')

        # Skip the first row of the csv
        next(reader)
        
        num_correct, num_guess, num_wrong = 0, 0, 0

        details_file = open(f"{model_name}-details.csv", mode='w', newline="")
        details_writer = csv.writer(details_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        analysis_file = open(f"analysis.csv", mode='a', newline="")
        analysis_writer = csv.writer(analysis_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
        for row in reader:
            question_word, answer_word, guess_word, similarity, label = None, None, None, None, None

            (question_word, answer_word, guess_word, similarity, label) = compute_synonym(model, row)
            
            if label == Labels.CORRECT:
                num_correct += 1
            elif label == Labels.GUESS:
                num_guess += 1
            else:
                num_wrong += 1

            output = f"{question_word},{answer_word},{guess_word},{label.value}"
            print(output)

            details_writer.writerow([question_word, answer_word, guess_word, label.value])
        
        analysis_writer.writerow([model_name, len(model), num_correct, (num_correct + num_wrong), num_correct/(num_correct + num_wrong)])
        print_analysis(model, model_name, num_correct, num_wrong)
        
        details_file.close()
        analysis_file.close()

def clear_file(file_name):
    analysis_file = open(f"{file_name}", mode='w')
    analysis_file.truncate(0)
    analysis_file.close()

clear_file("analysis.csv")
clear_file("word2vec-google-news-300-details.csv")
compute_synonyms(model_word2vec_google_news_300, "word2vec-google-news-300")

enormously,tremendously,tremendously,correct
provisions,stipulations,stipulations,correct
haphazardly,randomly,randomly,correct
prominent,conspicuous,conspicuous,correct
zenith,pinnacle,pinnacle,correct
flawed,imperfect,imperfect,correct
urgently,desperately,desperately,correct
consumed,eaten,eaten,correct
advent,coming,coming,correct
concisely,succinctly,succinctly,correct
salutes,greetings,ceremonies,wrong
solitary,alone,restless,wrong
hasten,accelerate,accelerate,correct
perseverance,endurance,generosity,wrong
fanciful,imaginative,imaginative,correct
showed,demonstrated,demonstrated,correct
constantly,continually,continually,correct
issues,subjects,subjects,correct
furnish,supply,impress,wrong
costly,expensive,expensive,correct
recognized,acknowledged,acknowledged,correct
spot,location,location,correct
make,earn,earn,correct
often,frequently,frequently,correct
easygoing,relaxed,relaxed,correct
debate,argument,argument,correct
narrow,thin,thin,correct
arranged,planned,planned,correct

# 2.3 Task 2: Comparison with Other Pre-trained Models

1. 2 new models from different corpora but same embedding size

In [5]:
# Let's use a model from Wikipedia corpora with embedding size of 100
model_word2vec_glove_wiki_100 = api.load("glove-wiki-gigaword-100")

# Let's use a model from Twitter corpora with embedding size of 100
model_word2vec_glove_twitter_100 = api.load("glove-twitter-100")

[==================================================] 100.0% 128.1/128.1MB downloaded
[==================================================] 100.0% 387.1/387.1MB downloaded


In [6]:
compute_synonyms(model_word2vec_glove_wiki_100, "glove-wiki-gigaword-100")
compute_synonyms(model_word2vec_glove_twitter_100, "glove-twitter-100")

enormously,tremendously,tremendously,correct
provisions,stipulations,stipulations,correct
haphazardly,randomly,randomly,correct
prominent,conspicuous,ancient,wrong
zenith,pinnacle,pinnacle,correct
flawed,imperfect,imperfect,correct
urgently,desperately,desperately,correct
consumed,eaten,eaten,correct
advent,coming,coming,correct
concisely,succinctly,succinctly,correct
salutes,greetings,greetings,correct
solitary,alone,restless,wrong
hasten,accelerate,accelerate,correct
perseverance,endurance,generosity,wrong
fanciful,imaginative,imaginative,correct
showed,demonstrated,demonstrated,correct
constantly,continually,continually,correct
issues,subjects,subjects,correct
furnish,supply,advise,wrong
costly,expensive,expensive,correct
recognized,acknowledged,successful,wrong
spot,location,location,correct
make,earn,earn,correct
often,frequently,frequently,correct
easygoing,relaxed,relaxed,correct
debate,argument,argument,correct
narrow,thin,thin,correct
arranged,planned,planned,correct
infinite,

2. 2 new models from the same corpus but different embedding sizes

In [8]:
# Let's use a model from Wikipedia corpora with embedding size of 50
model_word2vec_glove_wiki_50 = api.load("glove-wiki-gigaword-50")

# Let's also use a model from Wikipedia corpora with embedding size of 300 instead
model_word2vec_glove_wiki_300 = api.load("glove-wiki-gigaword-300")

[==================================================] 100.0% 66.0/66.0MB downloaded
[==================================================] 100.0% 376.1/376.1MB downloaded


In [ ]:
compute_synonyms(model_word2vec_glove_wiki_50, "glove-wiki-gigaword-50")
compute_synonyms(model_word2vec_glove_wiki_300, "glove-wiki-gigaword-300")